In [6]:
a = !pip list

In [8]:
with open('./version.txt', 'w') as f :
    f.writelines(a)

In [3]:
!python --version

Python 3.8.8


# 웹 크롤링

In [1]:
import time, re
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

import warnings
warnings.filterwarnings('ignore')

In [2]:
def date_range(start, end) :
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [(start + timedelta(days = i)).strftime("%Y%m%d") for i in range((end - start).days + 1)]
    return dates

In [3]:
def chrome_driver_options() :
    options = Options()
    
    # 사용하는 크롬 버전의 유저 에이전트로 수정
    
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36"
    options.add_argument('user-agent=' + user_agent)
    
    # 이미지 로딩을 하지 않음
    options.add_argument('--blink-settings=imagesEnabled=false') 
    # 음소거 적용
    options.add_argument('--mute-audio')
    # 시크릿 모드
    options.add_argument('incognito')
    
    return options

In [4]:
# 회사명으로 주식 종목 코드를 획득할 수 있도록 하는 함수
def get_code(name, daum_talk = 0) :
    
    code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header = 0)[0]
    code_df = code_df[['회사명', '종목코드']]
    
    # data frame title 변경 '회사명' = name, 종목코드 = 'code'
    code_df = code_df.rename(columns = {'회사명': 'name', '종목코드': 'code'})

    # 종목코드는 6자리로 구분되기때문에 0을 채워 6자리로 변경
    code_df.code = code_df.code.map('{:06d}'.format)
    target_code = code_df.query(f"name=='{name}'")
    code = target_code['code'].to_string(index = False)
    
    # 위와 같이 code명을 가져오면 앞에 공백이 붙어있는 상황이 발생하여 앞뒤로 sript() 하여 공백 제거
    fin_code = code.strip()
    
    # 다음 주식 검색을 위한 코드
    if daum_talk == 1 :
        fin_code = 'A' + code   
    
    return fin_code

In [5]:
driver_path = "./chromedriver.exe"
s = Service(driver_path)
cols = ['Date', 'Title']
codes = ['LG화학', '삼성SDI', 'SK이노베이션', '고려아연', '포스코케미칼']

In [15]:
DATE_LIST = date_range("20220802", "20220802")

## 1. 다음 주식 NEWS

In [16]:
def daum_Article_Scraping() :
    # 반환값으로 전달할 daum_news_df
    daum_news_df = pd.DataFrame(columns = cols)
    
    # 다음 주식 뉴스 페이지에 나타나는 기사의 최대 갯수
    NEWS_MAX_CNT = 15
    
    try :
        for date in DATE_LIST :
            # 크롤링 시작
            driver = webdriver.Chrome(service = s, options = chrome_driver_options())

            # 1페이지부터 접속
            page_cnt = 1
            past_li = []

            while True :
                li = []
                # 다음 주식 뉴스 페이지
                driver.get(f"https://news.daum.net/breakingnews/economic/stock?page={page_cnt}&regDate={date}")
                time.sleep(1)

                # css를 통해 기사 제목의 텍스트 추출
                articles = driver.find_elements(By.CSS_SELECTOR,'div.box_etc div > strong > a.link_txt')

                for i in articles :
                    li.append([date, i.text])

                # 이전 뉴스 페이지의 기사 제목과 현재 뉴스 페이지의 기사 제목이 같으면 탈출
                # 마지막 페이지인지 검사하는 조건문(1)
                if past_li == li :
                    break

                # 결과 데이터프레임과 합병할 데이터 프레임 생성
                temp_df = pd.DataFrame(li, columns = cols) 

                daum_news_df = pd.concat([daum_news_df, temp_df], ignore_index = True)

                # 기사 제목이 NEWS_MAX_CNT 보다 모자라면 탈출, 마지막 페이지인지 검사하는 조건문(2)
                if len(articles) < NEWS_MAX_CNT :
                    break

                # 현재의 기사 제목을 이전 기사 제목 리스트 변수에 저장
                past_li = li

                # 다음 페이지
                page_cnt += 1

            # 하루의 주식 기사들을 모두 확인했다면 종료
            driver.quit()
    except :
        print('check')
        driver.quit()
        pass

    # 반환 전 중복 제거
    daum_news_df.drop_duplicates(['Title'], keep = 'first', inplace = True)
    return daum_news_df

In [17]:
daum_news_df = daum_Article_Scraping()

In [18]:
daum_news_df.to_csv('./220802_daum_new.csv', encoding='utf-8-sig')

# 2. 다음 토론방

In [19]:
def daum_Talk_Scraping(input_code) :
    code = get_code(input_code, 1)
    
    dates = []
    titles = []
    
    start = datetime.strptime(DATE_LIST[0][:-2], "%Y%m") - timedelta(days = 1)
    start = str(start)[2: 7].replace('-','.') +'.'
    
    driver = webdriver.Chrome(service = s, options = chrome_driver_options())
    driver.get(f'https://finance.daum.net/quotes/{code}#talks')
    time.sleep(3)
    
    TALK_MAX_CNT = 31
    
    try :
        # start의 날짜를 찾을 때까지 반복
        while True :
            
            # 10페이지 중 현재 페이지를 제외한 9페이지 만큼 반복
            for i in range(9) :
                
                # 한 페이지에 0번째 글은 공지사항 고정,
                # 1번째부터 시작하여 30번째 글의 제목까지 반복
                for j in range(1, TALK_MAX_CNT) :
                    date = driver.find_elements(By.CSS_SELECTOR, '#boxContents > div.tableB tbody td:nth-child(5)')[j]
                    
                    # start의 날짜를 찾으면 전체 반복문 탈출
                    if re.search(fr'^{start}', date.text) :
                        raise NotImplementedError
                        
                    dates.append(date.text)
                    title = driver.find_elements(By.CSS_SELECTOR, '#boxContents > div.tableB tbody td:nth-child(1)')[j]

                    # 제목이 너무 길 경우 ...으로 끝남
                    if title.text.endswith('...') :
                        # 해당 글로 이동하여 제목 텍스트를 읽음
                        driver.find_element_by_link_text(title.text).click()
                        time.sleep(2)

                        element = driver.find_element(By.ID, "bbsFrame")
                        driver.switch_to.frame(element)
                        time.sleep(1)

                        title_in = driver.find_elements(By.CSS_SELECTOR, '#bbsTitle')[0]
                        titles.append(title_in.text)
                        
                        driver.back()
                        time.sleep(2)

                    else :
                        titles.append(title.text)
                        time.sleep(1)

                # 30줄의 토론방 제목을 전부 다 보았다면 다음 페이지로 이동
                driver.find_elements(By.CSS_SELECTOR, '.btnMove')[i].click()
                time.sleep(1)

            # 10페이지씩 전부 다 봤다면 다음 페이지 목록으로 이동
            driver.find_elements(By.CSS_SELECTOR, '.btnNext')[0].click()
            time.sleep(1)

    except NotImplementedError :
        print('find start month')
        driver.quit()
    except :
        print('check')
        pass

    daum_talk_df = pd.DataFrame(list(zip(dates, titles)), columns = cols)
    daum_talk_df.drop_duplicates(['Title'], keep = 'first', inplace = True)
    daum_talk_df['Date'] = '20' + daum_talk_df['Date']
    daum_talk_df.sort_values('Date', ignore_index = True, inplace = True)

    return daum_talk_df

In [20]:
### codes = ['LG화학', '삼성SDI', 'SK이노베이션', '고려아연', '포스코케미칼']
daum_talk_df = daum_Talk_Scraping(codes[2])

find start month


In [21]:
daum_talk_df

,Date,Title


In [14]:
daum_talk_df.to_csv('./220728_daum_talk.csv', encoding='utf-8-sig')

# 3. 네이버 주식 NEWS

- 네이버는 __뉴스 홈의 경제 증권 뉴스__ 와 __금융 홈의 기업 종목분석 뉴스__ 를 선택하였다.

### 뉴스 홈의 경제 증권 뉴스

In [22]:
def naver_News_Home_Article_Scraping() :
    naver_news_df = pd.DataFrame(columns = cols)

    NEWS_MAX_CNT = 20

    try :
        for date in DATE_LIST :
            driver = webdriver.Chrome(driver_path, options = chrome_driver_options())

            past_li = []
            page_cnt = 1

            while True :
                li = []
                # 네이버 뉴스 홈의 경제 증권 뉴스 페이지
                driver.get(f'https://news.naver.com/main/list.naver?mode=LS2D&sid2=258&sid1=101&mid=shm&date={date}&page={page_cnt}')
                time.sleep(1)
                
                articles = driver.find_elements(By.CSS_SELECTOR, f'#main_content > div > ul > li > dl > dt > a')

                for article in articles :
                    if len(article.text) != 0 :
                        li.append([date, article.text])

                if past_li == li :
                    break

                temp_df = pd.DataFrame(li, columns = cols)

                naver_news_df = pd.concat([naver_news_df, temp_df], ignore_index = True)

                if len(li) < NEWS_MAX_CNT :
                    break

                past_li = li
                page_cnt += 1

            driver.quit()
    except :
        print('check')
        pass

    naver_news_df.drop_duplicates(['Title'], keep = 'first', inplace = True)

    return naver_news_df

In [23]:
naver_news_df = naver_News_Home_Article_Scraping()

In [24]:
naver_news_df.query('Title != "동영상기사"').to_csv('./220802_naver_news_1.csv', encoding='utf-8-sig')

### 금융 홈의 기업 종목분석 뉴스 

In [25]:
def naver_Finance_Home_Article_Scraping() :
    naver_finance_df = pd.DataFrame(columns = cols)
    NEWS_MAX_CNT = 20

    try :
        for date in DATE_LIST :
            driver = webdriver.Chrome(driver_path, options = chrome_driver_options())

            past_li = []
            page_cnt = 1

            while True :
                li = []
                # 네이버 금융 홈의 기업 종목분석 뉴스 페이지
                driver.get(f'https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=402&date={date}&page={page_cnt}')
                time.sleep(1)

                articles = driver.find_elements(By.CSS_SELECTOR, '.articleSubject > a')

                for article in articles :
                    if len(article.text) != 0 :
                        li.append([date, article.text])

                if past_li == li :
                    break

                temp_df = pd.DataFrame(li, columns = cols)

                naver_finance_df = pd.concat([naver_finance_df, temp_df], ignore_index = True)

                if len(li) < NEWS_MAX_CNT :
                    break

                past_li = li
                page_cnt += 1

            driver.quit()
    except :
        print('check')
        driver.quit()
        pass
    
    naver_finance_df.drop_duplicates(['Title'], keep = 'first', inplace = True)

    return naver_finance_df

In [26]:
naver_finance_df = naver_Finance_Home_Article_Scraping()

In [27]:
naver_finance_df.query('Title != "동영상기사"').to_csv('./220802_naver_news_2.csv', encoding='utf-8-sig')

# 4. 네이버 토론방

In [28]:
def naver_Talk_Scraping(input_code) :
    code = get_code(input_code)
    
    # 찾고자 하는 연도와 월의 이전 월을 찾음
    start = datetime.strptime(DATE_LIST[0][:-2], "%Y%m") - timedelta(days = 1)
    start = str(start)[:7].replace('-','.') +'.'
    
    # 1페이지부터 접속
    page_cnt = 1
    dates = []
    titles = []

    try :
        driver = webdriver.Chrome(service = s, options = chrome_driver_options())
        while True :
            # 네이버 주식 토론방 페이지
            driver.get(f'https://finance.naver.com/item/board.naver?code={code}&page={page_cnt}')
            time.sleep(2)
            
            # css를 통해 제목과 날짜 추출
            date = driver.find_elements(By.CSS_SELECTOR, '#content > div.section.inner_sub > table.type2 > tbody > tr > td:nth-child(1) > span')
            title = driver.find_elements(By.CSS_SELECTOR,'#content > div.section.inner_sub > table.type2 > tbody > tr td.title > a')
            
            # 찾고자 하는 날짜보다 오래된 날짜를 찾으면 반복문 탈출
            for j in range(len(date)) :
                if re.search(fr'^{start}', date[j].text) :
                    raise NotImplementedError
                
                dates.append(date[j].text[2:10])
                titles.append(title[j].get_attribute('title'))
            
            # 다음 페이지
            page_cnt += 1

    except NotImplementedError :
        print('find start month')
        driver.quit()
    except :
        print('check')
        pass
    
    # 반환 전 중복 제거 및 정렬
    naver_talk_df = pd.DataFrame(list(zip(dates, titles)), columns = cols)
    naver_talk_df.drop_duplicates(['Title'], keep = 'first', inplace = True)
    naver_talk_df['Date'] = '20' + naver_talk_df['Date']
    naver_talk_df.sort_values('Date', ignore_index = True, inplace = True)
    
    return naver_talk_df

In [29]:
# codes = ['LG화학', '삼성SDI', 'SK이노베이션', '고려아연', '포스코케미칼']
naver_talk_df = naver_Talk_Scraping(codes[2])

check


In [30]:
naver_talk_df.query('Date == "2022.08.02"').to_csv('./220802_naver_talk.csv', encoding="utf-8-sig")

# 5. 유튜브 제목

In [36]:
def youtube_Scraping(input_code) :
    title = []
    day = []
    cnt = 0
    
    driver = webdriver.Chrome(service = s, options = chrome_driver_options())
    
    # 구글 페이지에서 유튜브 검색
    driver.get('https://www.google.com/')
    time.sleep(1)

    srart = f'{DATE_LIST[0][4:6]}/{DATE_LIST[0][6:]}/{DATE_LIST[0][:4]}'
    end = f'{DATE_LIST[-1][4:6]}/{DATE_LIST[-1][6:]}/{DATE_LIST[-1][:4]}'
    
    # 구글링 
    driver.find_element(By.CLASS_NAME, 'gLFyf.gsfi').send_keys(f"site:youtube.com {input_code}")
    driver.find_element(By.CLASS_NAME, 'gLFyf.gsfi').send_keys(Keys.ENTER)
    time.sleep(1)
    driver.find_element(By.CLASS_NAME, 't2vtad').click()
    time.sleep(1)
    driver.find_elements(By.CLASS_NAME, 'gTl8xb')[2].click()
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="lb"]/div/g-menu/g-menu-item[7]/div/div/span').click()
    time.sleep(1)
    driver.find_element(By.CLASS_NAME, 'OouJcb').send_keys(srart)
    driver.find_element(By.CLASS_NAME, 'rzG2be').send_keys(end)
    time.sleep(1)
    driver.find_element(By.CLASS_NAME, 'Ru1Ao.BwGU8e.fE5Rge').click()
    time.sleep(1)

    menu = driver.find_elements(By.CSS_SELECTOR, '#hdtb-msb > div:nth-child(1) > div > div > a')

    for i in range(len(menu)) :
        if menu[i].text == "동영상" :
            driver.find_element(By.XPATH, f'//*[@id="hdtb-msb"]/div[1]/div/div[{i + 2}]/a').click()
            break
    
            
    # 유튜브 제목 크롤링
    page_cnt = 0
    try :
        while True :
            channels = driver.find_elements(By.CLASS_NAME, 'LC20lb.MBeuO.DKV0Md')
            for channel in range(len(channels)) :

                if channel != 0 and channel % 4 == 0 :
                    ActionChains(driver).move_to_element(channels[channel]).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).perform()
                    time.sleep(1)
                channels[channel].click()
                time.sleep(2)

                title_txt = driver.find_element(By.XPATH, '//*[@id="container"]/h1/yt-formatted-string').text
                day_txt = driver.find_element(By.XPATH, '//*[@id="info-strings"]/yt-formatted-string').text
                title.append(title_txt)
                day.append(day_txt)
                cnt += 1

                driver.back()
                time.sleep(2)

            pages = driver.find_elements(By.CLASS_NAME, 'fl')

            pages[page_cnt].click()

            if page_cnt >= 10 :
                page_cnt = 10
                if len(pages) <= 10 :
                    break
            else :
                page_cnt += 1

            time.sleep(2)
    except :
        print('check')
        
    df = pd.DataFrame(list(zip(day, title)), columns = cols)
    df.sort_values('Date', ignore_index = True, inplace = True)
    
    return df

In [37]:
youtube_df = youtube_Scraping(codes[2])

check


In [38]:
youtube_df

,Date,Title
0,2022. 7. 28.,나스닥 급등! 코스피 향방은? ㅣ SK이노베이션 리노공업 주가전망


In [36]:
youtube_df.to_csv('./220725_youtube.csv', encoding="ANSI")

#  투자 결정 코드

In [6]:
import pickle
from konlpy.tag import *

def tokenizer(text):
    okt = Okt()
    return okt.morphs(text)

In [31]:
d_news = pd.read_csv('./220802_daum_new.csv', index_col=0, encoding='utf-8-sig')
#d_talk = pd.read_csv('./220726_daum_talk.csv', index_col=0, encoding='ANSI')
n_news_1 = pd.read_csv('./220802_naver_news_1.csv', index_col=0, encoding='utf-8-sig')
n_news_2 = pd.read_csv('./220802_naver_news_2.csv', index_col=0, encoding='utf-8-sig')
n_talk = pd.read_csv('./220802_naver_talk.csv' , index_col=0, encoding='utf-8-sig')
#youtube = pd.read_csv('./220723,24_youtube.csv', index_col=0, enconding='ANSI')

In [32]:
df = pd.concat([d_news, n_news_1, n_news_2 ,n_talk])
#df = pd.concat([d_news, d_talk, n_news_1, n_news_2 ,n_talk])

# 'Date' 타입이 int 이므로 datetime으로 변환
df['Date'] = pd.to_datetime(df['Date'].astype(str))

df.sort_values('Date', ignore_index = True, inplace = True)

In [33]:
df['Title'] = df['Title'].astype(str)
df.drop_duplicates('Title', ignore_index=True, inplace=True)

In [34]:
df['Date'].value_counts()

2022-08-02    943
Name: Date, dtype: int64

In [35]:
df.to_csv('220802_titles_for_investment_game.csv',encoding='utf-8-sig')

In [36]:
df = pd.read_csv('./220802_titles_for_investment_game.csv', encoding='utf-8', index_col=0)
print(len(df))

943


In [7]:
pipeline = './[Model5]pipe_binary.dat'
with open(pipeline,'rb') as fp:     # 읽기
    pipe = pickle.load(fp)

In [38]:
score = []
for i in tqdm(range(len(df))) :
    score.append(pipe.predict([df['Title'][i]])[0])

100%|███████████████████████████████████████████████████████████████████████████████| 943/943 [00:05<00:00, 186.89it/s]


In [39]:
df['score'] = score

In [40]:
print(df['score'].value_counts())

 1    690
-1    253
Name: score, dtype: int64


In [8]:
temp = pd.read_csv('220804_titles_for_investment_game.csv', encoding='utf-8-sig', index_col=0)

In [9]:
len(temp)

737

In [10]:
temp.drop_duplicates('Title', ignore_index=True, inplace=True)

In [11]:
len(temp)

737

In [12]:
score = []
for i in tqdm(range(len(temp))) :
    score.append(pipe.predict([temp['Title'][i]])[0])

100%|████████████████████████████████████████████████████████████████████████████████| 737/737 [00:14<00:00, 52.16it/s]


In [13]:
temp['score'] = score
temp['score'].value_counts()

 1    606
-1    131
Name: score, dtype: int64